In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame
import matplotlib.pyplot as plt 
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
# from sklearn.datasets import make_classification
import seaborn as sns

In [2]:
train=pd.read_csv("Training Data.csv")
train=train.drop(columns=["Id"]) # I have drop "Id" column because it doesn't represents any feature
test=pd.read_csv("Test Data.csv")
t=test
test=test.drop(columns=["id"])



In [3]:
train.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [4]:
is_NaN = train.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train[row_has_NaN]

print(rows_with_NaN)

Empty DataFrame
Columns: [income, age, experience, married, house_ownership, car_ownership, profession, city, state, current_job_years, current_house_years, risk_flag]
Index: []


In [5]:
print(train.house_ownership.value_counts())
print("..................")
print(train.car_ownership.value_counts())
print("..................")
print(train.profession.value_counts().describe())


rented          231898
owned            12918
norent_noown      7184
Name: house_ownership, dtype: int64
..................
no     176000
yes     76000
Name: car_ownership, dtype: int64
..................
count      51.000000
mean     4941.176471
std       374.277154
min      4048.000000
25%      4670.000000
50%      4881.000000
75%      5200.000000
max      5957.000000
Name: profession, dtype: float64


In [6]:
house_ownership=pd.get_dummies(data=train['house_ownership'], columns=['house_ownership'])
risk_flag=train.loc[:,["risk_flag"]]

In [7]:
house_ownership.head()

,norent_noown,owned,rented
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [8]:
risk_flag.head()

,risk_flag
0,0
1,0
2,0
3,1
4,1


In [9]:
from sklearn.feature_selection import chi2
f_p_values=chi2(house_ownership,risk_flag)

In [10]:
p_values=pd.Series(f_p_values[1])
print(f_p_values)
p_values.index=house_ownership.columns
p_values.sort_values(ascending = True)

(array([ 36.69517416, 132.02022214,  14.27384509]), array([1.38120934e-09, 1.48102818e-30, 1.58045758e-04]))


owned           1.481028e-30
norent_noown    1.381209e-09
rented          1.580458e-04
dtype: float64

In [11]:
train['house_ownership']=np.where(train['house_ownership']=='owned',0,1)
test['house_ownership']=np.where(test['house_ownership']=='owned',0,1)

In [12]:
print(train.house_ownership.value_counts())

1    239082
0     12918
Name: house_ownership, dtype: int64


In [13]:

# train.head()
train["married"]=pd.factorize(train.married)[0]
test["married"]=pd.factorize(test.married)[0]
# train["house_ownership"]=pd.factorize(train.house_ownership)[0]
train["car_ownership"]=pd.factorize(train.car_ownership)[0]
test["car_ownership"]=pd.factorize(test.car_ownership)[0]

In [14]:
# train=pd.get_dummies(data=train, columns=['profession'])
# test=pd.get_dummies(data=test, columns=['profession'])

In [15]:
len(train['profession'].value_counts().index)

51

In [16]:
len(test['profession'].value_counts().index)

51

In [17]:
# plt.figure(figsize=(30,30))
# correlation_mat = train.corr()

# sns.heatmap(correlation_mat, annot = True)

# plt.show()


In [18]:
# train["exp_jobyear"] = train['experience']-train['current_job_years']
# test["exp_jobyear"] = test['experience']-test['current_job_years']
# train["house_rented_norent"]=train['house_ownership_norent_noown']/train['house_ownership_rented']
# train["house_rented_owned"]=train['house_ownership_owned']/train['house_ownership_rented']

train=train.drop(["current_job_years"],axis=1)
test=test.drop(["current_job_years"],axis=1)

In [19]:
# plt.figure(figsize=(15, 15))
# correlation_mat = train.corr()

# sns.heatmap(correlation_mat, annot = True)

# plt.show()

In [20]:
state=pd.get_dummies(data=train['state'], columns=['state'])
f_p_values=chi2(state,risk_flag)
p1_values=pd.Series(f_p_values[1])
p1_values.index=state.columns
print(p1_values.sort_values(ascending = True).head(10))
print(p1_values.describe())

Madhya_Pradesh    7.412887e-30
Kerala            1.460364e-24
Manipur           2.203549e-16
Tamil_Nadu        8.028357e-15
Karnataka         5.196389e-14
Punjab            5.422502e-12
Uttarakhand       6.999429e-12
Sikkim            7.609934e-09
Rajasthan         1.987381e-07
Maharashtra       2.092608e-06
dtype: float64
count    2.900000e+01
mean     9.128890e-02
std      1.916380e-01
min      7.412887e-30
25%      7.609934e-09
50%      7.265550e-04
75%      5.258925e-02
max      7.439216e-01
dtype: float64


In [21]:
city=pd.get_dummies(data=train['city'], columns=['city'])
f_p_values=chi2(city,risk_flag)
p2_values=pd.Series(f_p_values[1])
p2_values.index=city.columns
# print(p2_values.sort_values(ascending = True).head())
# print(p2_values.describe())

In [22]:
downstate=[x for x in p1_values.sort_values(ascending = True).index[9:30]]
downcity=[x for x in p2_values.sort_values(ascending = True).index[43:318]]

downstate_test=[x for x in p1_values.sort_values(ascending = True).index[9:30]]
downcity_test=[x for x in p2_values.sort_values(ascending = True).index[43:318]]

In [23]:
# downstate

In [24]:
for i in range(20):
    train['state']=train['state'].replace(downstate[i],'other_state')#[ ,,]
    train['state']=train['state'].replace('Kerala','other_state')
    train['state']=train['state'].replace('Manipur','other_state')
    train['state']=train['state'].replace('Uttarakhand','other_state')
for j in range(274):
    train['city']=train['city'].replace(downcity[j],'other_city')

In [25]:
for i in range(20):
    test['state']=test['state'].replace(downstate[i],'other_state')#[ ,,]
    test['state']=test['state'].replace('Kerala','other_state')
    test['state']=test['state'].replace('Manipur','other_state')
    test['state']=test['state'].replace('Uttarakhand','other_state')
for j in range(274):
    test['city']=test['city'].replace(downcity[j],'other_city')

In [26]:
train.head()
    

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_house_years,risk_flag
0,1303835,23,3,0,1,0,Mechanical_engineer,other_city,Madhya_Pradesh,13,0
1,7574516,40,10,0,1,0,Software_Developer,other_city,other_state,13,0
2,3991815,66,4,1,1,0,Technical_writer,other_city,other_state,10,0
3,6256451,41,2,0,1,1,Software_Developer,Bhubaneswar,other_state,12,1
4,5768871,47,11,0,1,0,Civil_servant,other_city,Tamil_Nadu,14,1


In [27]:
test.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_house_years
0,7393090,59,19,0,1,0,Geologist,other_city,West Bengal,13
1,1215004,25,5,0,1,0,Firefighter,other_city,other_state,10
2,8901342,50,12,0,1,0,Lawyer,other_city,other_state,14
3,1944421,49,9,1,1,1,Analyst,Latur,other_state,12
4,13429,25,18,0,1,1,Comedian,other_city,West Bengal,11


In [28]:
# state=pd.get_dummies(data=train['state'], columns=['state'])
# f_p_values=chi2(state,risk_flag)
# p3_values=pd.Series(f_p_values[1])
# p3_values.index=state.columns
# print(p3_values.sort_values(ascending = True).head(10))

In [29]:
# city=pd.get_dummies(data=train['city'], columns=['city'])
# f_p_values=chi2(city,risk_flag)
# p4_values=pd.Series(f_p_values[1])
# p4_values.index=city.columns
# print(p4_values.sort_values(ascending = True))

In [30]:
# train.state.value_counts().sort_values(ascending=False) #['state_Karnataka', 'state_Kerala', 'state_Madhya_Pradesh', 'state_Manipur', 'state_Rajasthan', 'state_Uttarakhand']

In [31]:
# train.city.value_counts().sort_values(ascending=False)

In [32]:
# train.describe()

In [33]:
len(train['city'].value_counts().index)

44

In [34]:
len(test['city'].value_counts().index)

58

In [35]:
len(train['state'].value_counts().index)

7

In [36]:
len(test['state'].value_counts().index)

13

In [37]:
train['city']=pd.get_dummies(train.city)
train['state']=pd.get_dummies(train.state)
train['profession']=pd.get_dummies(train.profession)
test['city']=pd.get_dummies(test.city)
test['state']=pd.get_dummies(test.state)
test['profession']=pd.get_dummies(test.profession)

In [38]:
test.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_house_years
0,7393090,59,19,0,1,0,0,0,0,13
1,1215004,25,5,0,1,0,0,0,0,10
2,8901342,50,12,0,1,0,0,0,0,14
3,1944421,49,9,1,1,1,0,0,0,12
4,13429,25,18,0,1,1,0,0,0,11


In [39]:
train.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_house_years,risk_flag
0,1303835,23,3,0,1,0,0,0,0,13,0
1,7574516,40,10,0,1,0,0,0,0,13,0
2,3991815,66,4,1,1,0,0,0,0,10,0
3,6256451,41,2,0,1,1,0,0,0,12,1
4,5768871,47,11,0,1,0,0,0,0,14,1


In [40]:
# train.head()

In [41]:
# train.describe()

In [42]:
xL_train=train.drop(["risk_flag"],axis=1)

# xL_test=test.drop(["city","state"],axis=1)
yL_train=train.loc[:,["risk_flag"]]

In [43]:
xL_train.head(5)

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_house_years
0,1303835,23,3,0,1,0,0,0,0,13
1,7574516,40,10,0,1,0,0,0,0,13
2,3991815,66,4,1,1,0,0,0,0,10
3,6256451,41,2,0,1,1,0,0,0,12
4,5768871,47,11,0,1,0,0,0,0,14


In [44]:
test.head()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_house_years
0,7393090,59,19,0,1,0,0,0,0,13
1,1215004,25,5,0,1,0,0,0,0,10
2,8901342,50,12,0,1,0,0,0,0,14
3,1944421,49,9,1,1,1,0,0,0,12
4,13429,25,18,0,1,1,0,0,0,11


In [45]:
# xL_test.head()

In [46]:
yL_train.head()

,risk_flag
0,0
1,0
2,0
3,1
4,1


In [47]:
X=xL_train.values
Y=yL_train.values
T=test.values

In [48]:
# X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))

# T= preprocessing.StandardScaler().fit(T).transform(T.astype(float))

In [49]:
X

array([[1303835,      23,       3, ...,       0,       0,      13],
       [7574516,      40,      10, ...,       0,       0,      13],
       [3991815,      66,       4, ...,       0,       0,      10],
       ...,
       [4522448,      46,       7, ...,       0,       0,      12],
       [6507128,      45,       0, ...,       0,       0,      10],
       [9070230,      70,      17, ...,       0,       0,      11]],
      dtype=int64)

In [50]:
Y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [51]:
from sklearn.tree import DecisionTreeClassifier
DTClassifier= DecisionTreeClassifier(criterion='entropy', random_state=42)
DTClassifier.fit(X,Y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [52]:
y_pred=DTClassifier.predict(T)
y_pred=pd.DataFrame(y_pred)
y_pred.columns =['risk_flag']
df=t.loc[:,["id"]]
Prediction_Dataset=pd.concat([df,y_pred,],axis=1).reset_index(drop="Ture")
Prediction_Dataset.to_csv('Dataset.csv',index=False)

In [53]:
X_test=pd.read_csv("X_test.csv")
Y_test=pd.read_csv("Y_test.csv")

In [54]:
X_test=X_test.values
Y_test=Y_test.values

In [55]:
X_test

array([[ 351361,      45,       5, ...,       0,       0,      12],
       [3553720,      49,       4, ...,       0,       0,      13],
       [7571507,      74,      17, ...,       0,       0,      12],
       ...,
       [7716196,      27,       5, ...,       0,       0,      14],
       [5218820,      76,      14, ...,       0,       0,      14],
       [ 296613,      23,       3, ...,       0,       0,      13]],
      dtype=int64)

In [56]:
y_hat=DTClassifier.predict(X_test)

In [57]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_hat,Y_test)

0.8481114043767168

In [58]:
#0.8481114043767168#0.8480986116406499